In [ ]:
import csv
import pandas as pd
import nltk
import numpy as np
import os
import unidecode

In [ ]:
######### EXTRACTING REPORTS FROM DATA LAKE

'''


SELECT patient_id, accession_number, exame_codigo, exame_desc, CAST(DATE(exam_datetime) AS DATE) AS exam_date, laudo_txt
FROM `interoper-dataplatform-prd.raw_rdi.rdi_tb_carestream_laudos` 
WHERE DATE(exam_datetime) BETWEEN "2021-01-01" AND "2021-06-18"
      
      AND (exame_desc LIKE '%RM%' OR exame_desc LIKE '%RESSONANCIA%')  
      
      AND exame_desc LIKE '%CRANIO%' 
      AND (exame_desc NOT LIKE '%ANGIO%')
      
    
'''

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')
include_stopwords = ['com','sem']
stopwords = [word for word in stopwords if not word in include_stopwords]

In [ ]:
# show stopwords
sorted(stopwords)

In [ ]:
#report_file = 'RM_CRANIO_JAN_JUN.csv'
report_file = 'RM_ABDOME_JAN_JUN.csv'
df = pd.read_csv(report_file, delimiter=',')
df

In [ ]:
drop_columns = [ 'exame_codigo', 'exame_desc']
df = df.drop(columns=drop_columns)
df

In [ ]:
def limpa_texto(texto: str):
    # tira a acentuacao do texto. Coração -> Coracao
    texto = unidecode.unidecode(texto)
    
    # formata o texto para evitar espacos, paragrafos, etc a mais
    texto = replace_string(texto, '*', '')
    texto = replace_string(texto, '..', '.')
    texto = replace_string(texto, '  ', ' ')
    texto = replace_string(texto, ' .', '.')
    texto = replace_string(texto, ' ,', ',')
    texto = replace_string(texto, '( ', '(')
    texto = replace_string(texto, ' )', ')')
    texto = replace_string(texto, ' :', ':')
    texto = replace_string(texto, '\\n', '\n')
    texto = replace_string(texto, '\t', '')
    texto = replace_string(texto, '\n ', '\n')
    texto = replace_string(texto, ' \n', '\n')
    texto = replace_string(texto, '\n.', '\n')
    texto = replace_string(texto, '\n\n\n', '\n\n')
    texto = texto.strip()
    texto = texto.strip('\n')
    
    # remove o nome do médico radiologista
    texto = remove_doctor_name(texto)
    
    return texto

In [ ]:
def replace_string(texto: str, procurado, substituido: str):
    # procura e troca as strings
    
    # faz a troca caso o parametro passado for uma string
    if type(procurado) is str:
        while procurado in texto:
            texto = texto.replace(procurado, substituido)
        return texto
    
    # faz a troca caso o parametro passado for uma lista
    elif type(procurado) is list:
        for palavra in procurado:
            while palavra in texto:
                texto = texto.replace(palavra, substituido)
        return texto

In [ ]:
def remove_doctor_name(texto: str):
    # palavras a serem buscadas no texto
    doctor_words = ['\n|Dr.', '\n|Dra.', "\nDr.", "\nDra."]
    
    # sai procurando pelas palavras de Dr(a).
    for word in doctor_words:
        if word in texto:
            # se encontrou, localizada a posicao
            pos = texto.find(word)
            # faz um slice no texto
            texto = texto[:pos]
    
    return texto        

In [ ]:
def lista_frases(texto: str, lista_frases_procuradas: list):
    # gera uma lista de frases quebradas por paragrafo
    
    lista_return = []
    
    # splita o texto por paragrafo
    frases_texto = texto.split('\n')
    
    for frase in frases_texto:
        for procurada in lista_frases_procuradas:
            # se achou o texto procurado na frase
            if procurada.upper() in frase.upper():
                # adiciona o paragrafo na lista de paragrafos encontrados
                lista_return.append(frase)
                break
    
    if lista_return == []:
        return ''
    else:
        return ' | '.join(lista_return)

In [ ]:
def remove_frases(texto: str, lista_frases_procuradas: list):
    
    lista_return = []
    # Tira o texto da Técnica
    texto = texto.replace('Tecnica:', '')
    
    # quebra as frases em paragrafos
    texto = texto.replace('. ', '\n')
    
    # splita o texto em paragrafos
    texto = texto.split('\n')
    
    for frase in texto:
        # itera cada frase do texto
        flag_remover = False
        for procurada in lista_frases_procuradas:
            if procurada.upper() in frase.upper():
                # se achou, marca como uma frase a remover
                flag_remover = True
                break
        
        # inclui na lista de frases, apenas aquelas que nao foram marcadas como remover
        if not flag_remover:
            lista_return.append(frase)
    
    # remonta o laudo apenas com as frases que nao foram removidas
    lista_return = '.\n'.join(lista_return)
    lista_return = limpa_texto(lista_return)
    
    return lista_return

In [ ]:
def contem_frase(texto: str, lista_frases_procuradas: list):
    # checa se uma frase está contida numa lista de frases
    for procurada in lista_frases_procuradas:
        if procurada.upper() in texto.upper():
            return True
                
    return False

In [ ]:
def remove_stopwords(texto: str):
    # garante que todo o conteúdo passado está em formato de lista
    if type(texto) is str:
        lista_texto = [texto]
    elif type(texto) is list:
        lista_texto = texto
    else:
        print("A função remove_stopwords aceita como parâmetro apenas strings ou listas.")
        exit()
    
    lista_removida = []        
    for text in lista_texto:
        # tira a acentuacao
        text = unidecode.unidecode(text)
        # tokeniza o texto tirando as stop words
        text_tokens = nltk.tokenize.word_tokenize(text, language='portuguese') 
        tokens_without_sw = [word for word in text_tokens if not word in stopwords]
        tokens_without_sw = ' '.join(tokens_without_sw)
        tokens_without_sw = limpa_texto(tokens_without_sw)
        # insere a frase limpa na lista frases sem stopwords
        lista_removida.append(tokens_without_sw)
    
    # reconverte a saída para o mesmo formato da entrada
    if type(texto) is str:
        return lista_removida[0]
    elif type(texto) is list:
        return lista_removida
    

In [ ]:
# define qual o nome da coluna do dataframe que contém o laudo
campo_laudo = 'laudo_txt'

# aplica a funcao limpa_texto para cada laudo de cada linha do dataframe
df[campo_laudo] = df.apply(lambda row: limpa_texto(row[campo_laudo]), axis = 1)

In [ ]:
df['laudo_txt'][0]

In [ ]:
# cria uma coluna com os laudos tokenizados. 
# essa funcao pode demorar alguns segundos para processar....

# select the columns name containig the report text
campo_laudo = 'laudo_txt'
df['laudo_tokenizado'] = df.apply(lambda row: remove_stopwords(row[campo_laudo]), axis = 1) 

In [ ]:
df['laudo_tokenizado'][0]

In [ ]:
dict_frases = {}

In [ ]:
# define as listas de inclusao e exclusao para detecao de contraste
frases_inclusao = [
                   'gadol', 'paramag', 'contrast'
                  ]

frases_exclusao = [ 'impregnacao', 
                    'nao administracao',
                    'nao autoriz',
                    'Nao foi administrado',
                    'Nao foi autoriz',
                    'nao foi injetado', 
                    'Nao foi obtida autorizacao',                  
                    'nao foi realizad', 
                    'nao foi utiliz', 
                    'Nao houve autorizacao',
                    'realce', 
                   'capta',
                   'reforc',
                    'realizado sem', 
                    'respeitando a opcao formal',
                    'sem administracao',
                    'sem contraste',
                    'sem o uso',
                   'sem injecao' 
                  ]

dict_frases['contraste'] = {"inclusao": frases_inclusao, "exclusao": frases_exclusao}

In [ ]:
# frases de inclusao e exclusao para glioma
frases_inclusao = ['glioma', 'gbm', 'neoplasia', 'glioblastoma', 'glial']

frases_exclusao = [ 'mening', 'menigo',
                   'p.o', 'cirur', 'operat',
                   'Schwannoma',
                   'mama', 'pulm', 'prost', 'renal', 'laringe', 'colo', 'ovari', 'intest', 'tiroid', 'tireo', 'oral', 'faringe','paragang', 'uter', 'gastr', 'estomago', 'mediast',
                   'secund', 'metast',
                   'Nao ha evidencia', 'Nao ha sugestao'
                  ]

dict_frases['glioma'] = {"inclusao": frases_inclusao, "exclusao": frases_exclusao}

In [ ]:
# frases de inclusao e exclusao para hcc
frases_inclusao = ['hcc', 'chc', 'hepatocelular', 'hepato-celular', 'hepato celular', 'carcinoma']

frases_exclusao = [ 'pancreas', 'renais', 'celulas claras', 'histori', 'hiperplasia', 'nefro', 'carcinomatos', 'adenocarcinoma',
                   'secund', 'metast', 'colangio', 'hemangioma', 'hnf', 'uro', 'ureter', 'hepatocelular benigna', 'hepatocelulares benignas', 'tireoide', 'ovario',
                   'reto', 'endometrio', 'vagina', 'uter',
                   'Nao ha', 'Nao caracterizam', 'Nao caracterizadas', 'Nao concentracao', 'Nao evidenciam',
                   'Nao ha esteatose, sobrecarga ferrica lesoes suspeitas hepatocarcinoma',
                   'Nao ha nodulos suspeitos hepatocarcinomas', 'nao observam', 'Nao caracterizado', 'Nao evidenciad', 'Nao surgi',
                   'Nao ha esteatose, sobrecarga ferrica lesoes suspeitas hepatocarcinoma', 
                   'Ausencia les', 'sem lesoes suspeitas','Ausencia outras lesoes suspeitas CHC',
                   'rastreio'
                  ]
dict_frases['hcc'] = {"inclusao": frases_inclusao, "exclusao": frases_exclusao}

In [ ]:
# defina na linha abaixo qual dicionario deseja procurar
dict_interesse = 'hcc'

frases_inclusao = remove_stopwords(dict_frases[dict_interesse]["inclusao"])
frases_exclusao = remove_stopwords(dict_frases[dict_interesse]["exclusao"])

In [ ]:
dict_frases[dict_interesse]

In [ ]:
# cria uma nova coluna contendo o laudo sem as frases de exclusao
df['laudo_pos_exclusao'] = df.apply(lambda row: remove_frases(row['laudo_tokenizado'], frases_exclusao), axis = 1)

In [ ]:
df['laudo_pos_exclusao'][0]

In [ ]:
# cria uma nova coluna contendo apenas as frases de inclusao que foram encontradas no laudo
df['frases_inclusao'] = df.apply(lambda row: lista_frases(row['laudo_pos_exclusao'], frases_inclusao), axis = 1) 
df['frases_inclusao'][0]

In [ ]:
for _ in list(df['frases_inclusao'].drop_duplicates().sort_values(ignore_index = True)):
    print(_)

In [ ]:
drop_columns = ['laudo_tokenizado', 'laudo_pos_exclusao']
df_export = df.drop(columns=drop_columns)

novo_nome_arq, _ = os.path.splitext(report_file) 
novo_nome_arq += "_NEW2.csv"
df_export.to_csv(novo_nome_arq, index = False)